In [1]:
import os, sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [2]:
import time
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import json
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import string
from itertools import islice
from selenium.webdriver.common.keys import Keys
# from Feed import Feed
from icecream import ic
from typing import Iterator, Any, List

In [3]:
from Surfer import GetBrowser,RedditFeed,Feed
from Surfer.ScriptRunner import ScriptRunner

In [4]:

def read_file_in_chunks(file_path, chunk_size=10):
    with open(file_path, 'rb') as file:
        while True:
            chunk = file.read(chunk_size)
            if not chunk:
                break
            
            binary_string = ''.join(format(byte, '08b') for byte in chunk)
            yield binary_string

# Example usage
ex = "svg"
file_path = f'tiny.{ex}'
data = ''
for chunk in read_file_in_chunks(file_path):
    data += chunk

# len(acc) - 796*8

def write_binary_string_to_file(binary_string, output_file_path):
    with open(output_file_path, 'wb') as file:
        for i in range(0, len(binary_string), 8):
            byte = binary_string[i:i + 8]
            if len(byte) < 8:
                byte = byte + '0' * (8 - len(byte))
            file.write(bytes([int(byte, 2)]))

# Example usage
binary_string = data
output_file_path = f'out/output_file.{ex}'
write_binary_string_to_file(binary_string, output_file_path)


In [5]:
import Stegasus 

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nasoo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Setting up Masked Stego



Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Setting up Masked Stego Completed


Setting up Typoceros

('hi, how atre you?', '01001011111')

Setting up Typoceros Completed



In [6]:
from Bot import *
from Chatter import prompts

In [7]:
driver = GetBrowser()

In [8]:
def uuid():
    letters_digits = string.ascii_letters
    return ''.join(random.choice(letters_digits) for i in range(20))

In [9]:
def N(
  mean = 0.5
	, stddev = 0.01
	, bounds = None
):
	if bounds:
		return max(bounds[0], min(random.normalvariate(mean, stddev) , bounds[1]))
	else:
		return random.normalvariate(mean, stddev)

In [10]:
run_script = ScriptRunner(driver,'./Surfer/scripts/')

In [11]:
class RedditPostFeed(Feed):
	def __init__(self, driver:webdriver.Chrome ,handle:str|None=None,verbose=False):
		super().__init__(name='Reddit Post Feed',driver=driver,handle=handle)
		# Store the WebDriver instance
		self.verbose = verbose
		self.focus()
		self.run_script('surfer.listen to clicks')()
		self.run_script('reddit.catch first comment',self.verbose)()

	def __iter__(self) -> Iterator[Any]:
		while True:
			self.focus()
			if not self.run_script('surfer.check cursor',verbose=self.verbose)():
				self.run_script('reddit.post.catch first comment',verbose=self.verbose)()
				continue
			self.run_script('surfer.scroll 2 cursor',verbose=self.verbose)(N(bounds=[0,1]),N(bounds=[0,1]),random.randint(1,25)/100)
			for i in range(10):
				if self.run_script('surfer.check still scrolling')():
					time.sleep(0.1)
				else:
					break
				if i == 9:
					raise Exception('Not finished scrolling')
			yield self.run_script('reddit.post.parse comment',verbose=self.verbose)()
			self.run_script('reddit.post.cursor next')()

class RedditFeed(Feed):
	def __init__(self, driver:webdriver.Chrome,skip=True,handle:str|None=None,verbose=False):
		super().__init__(name='Reddit Feed',handle=handle,driver=driver)
		self.verbose = verbose
		if not skip:
			self.driver.get('https://www.reddit.com/')
			self.handle = self.driver.current_window_handle
			self.run_script('surfer.listen to clicks')()
		self.run_script('reddit.init_reddit',verbose=self.verbose)()
	

	def __iter__(self) -> Iterator[Any]:
		while True:
			self.focus()
			if not self.run_script('surfer.check cursor',verbose=self.verbose)():
				continue
			self.run_script('surfer.scroll 2 cursor',verbose=self.verbose)(N(bounds=[0,1]),N(bounds=[0,1]),random.randint(1,25)/100)
			for i in range(10):
				if self.run_script('surfer.check still scrolling')():
					time.sleep(0.1)
				else:
					break
				if i == 9:
					raise Exception('Not finished scrolling')
			yield self.run_script('reddit.parse post',verbose=self.verbose)()
			self.run_script('surfer.cursor next')()

	def click_post(self, newTab=True):
		for i in range(10): # 10 retries
			self.focus()
			if i == 9:
				raise Exception('cursor not present')
			if not self.run_script('surfer.check cursor',verbose=self.verbose)():
				continue
			else:
				break
  
		id = uuid()
		self.run_script('surfer.mark cursor', verbose=self.verbose)(id)
		# Find the DOM element to click on
		sel = f'.{id}'
		element = driver.find_element(By.CSS_SELECTOR, sel)
		h3 = element.find_element(By.CSS_SELECTOR,'h3')
		element = element if h3 is None else h3
		actions = ActionChains(driver)
		actions \
			.click(element) \
			.perform()
			# .key_down(Keys.CONTROL) \
			# .key_up(Keys.CONTROL) \
		return RedditPostFeed(self.driver,self.handle,verbose=self.verbose)

	def close(self):
		self.focus()
		# Close the WebDriver instance
		self.driver.quit()


In [12]:
# rf = RedditFeed(driver,skip=False)
# rf = RedditFeed(driver,skip=True)

In [13]:
# for x in islice(rf,15):
#   print(x)
#   time.sleep(2*max(1,len(x) * 0.01))

In [14]:
# e = rf.click_post()

In [15]:
# run_script('reddit.post.parse comment')()

In [34]:
from typing import Tuple


class ChatGPTWebsite:
	def __init__(self, driver:webdriver.Chrome,skip=True,handle:str|None=None,verbose=False):
		self.name = 'GPT'
		self.handle = handle
		self.driver = driver
		self.verbose = verbose
		self.run_script = ScriptRunner(driver,'./Surfer/scripts/')
		if not skip:
			self.driver.get('https://chat.openai.com/')
			self.handle = self.driver.current_window_handle
			self.run_script('surfer.listen to clicks')()
	def getLandingNewChat(self):
		return ChatGPT(self.driver)

class ChatGPT:
	def __init__(self, driver:webdriver.Chrome, name:str|None = None):
		self.driver = driver
		self.run_script = ScriptRunner(driver,'./Surfer/scripts/')
		self.name = name

	def ask(self,prompt:str, newChat:bool = False) -> Tuple[str, 'ChatGPT']:
		if newChat:
			self.run_script('chat.openai.new chat')()
			WebDriverWait(self.driver, 30).until(
				EC.presence_of_element_located((By.CSS_SELECTOR, 'svg > circle+line+line+line+line+line+line+line+line'))# examples sun in new chat
			)
		else:
			if self.name:
				self.run_script('chat.openai.switch to chat name')(self.name)
				time.sleep(4)
		textarea = self.driver.find_element(By.CSS_SELECTOR,'textarea')

		JS_ADD_TEXT_TO_INPUT = """
			var elm = document.querySelector('textarea'), txt = arguments[0];
			elm.value += txt;
			elm.dispatchEvent(new Event('change'));
			"""
		prompts = prompt.split('\n')
		for line in prompts:
			self.driver.execute_script(JS_ADD_TEXT_TO_INPUT, line)
			textarea.send_keys(Keys.SHIFT + Keys.ENTER)
		time.sleep(2)
		self.driver.find_element(By.CSS_SELECTOR,'textarea + button').click()
		svg_path = 'M20.49 9A9 9 0 0 0 5.64 5.64L1 10m22 4l-4.64 4.36A9 9 0 0 1 3.51 15'
		try:
			WebDriverWait(self.driver, 60).until(
				EC.presence_of_element_located((By.CSS_SELECTOR, f'[d="{svg_path}"]'))
			)
		except:
			input('>>> Bot check please notify when done!...')
			WebDriverWait(self.driver, 60).until(
				EC.presence_of_element_located((By.CSS_SELECTOR, f'[d="{svg_path}"]'))
			)
		time.sleep(5)
		if self.name is None: # first ChatGPT instance ever or new is pressed and we are already at new
			self.name = self.run_script('chat.openai.get current chat name')()
			return self.run_script('chat.openai.get last answer')(), self
		elif newChat:
			name = self.run_script('chat.openai.get current chat name')()
			return self.run_script('chat.openai.get last answer')(), ChatGPT(self.driver,name)
		else:
			return self.run_script('chat.openai.get last answer')(), self

In [35]:
gptw = ChatGPTWebsite(driver,skip=False)

In [36]:
alice = Person('alice','nice',23,'female','Boston','alice@gmail.com')
bob = Person('bob','wick',29,'male','Boston','bob@gmail.com')

In [37]:
actualChat = []

In [38]:
data

'001111000111001101110110011001110010000001111000011011010110110001101110011100110011110100100010011010000111010001110100011100000011101000101111001011110111011101110111011101110010111001110111001100110010111001101111011100100110011100101111001100100011000000110000001100000010111101110011011101100110011100100010001000000110011001101001011011000110110000111101001000100110111001101111011011100110010100100010001000000111011001101001011001010111011101000010011011110111100000111101001000100011000000100000001100000010000000110010001101000010000000110010001101000010001000100000011100110111010001110010011011110110101101100101001011010111011101101001011001000111010001101000001111010010001000110001001011100011010100100010000011010000101000001001011100110111010001110010011011110110101101100101001111010010001001100011011101010111001001110010011001010110111001110100010000110110111101101100011011110111001000100010001000000110000101110010011010010110000100101101011010000110100101100100011001000110010

In [39]:
demo_post = '''Hi there! recently unstaked IMX from the staking L2 IMX plattform. And withdrawed from L2 to L1 on the Immutable marketplace, as indicated. It says there the withdrawal process can take up to 48 hours. BUT it is taking much more than that. 4 days and still nothing. Any hints?

thanks for any info related guys'''

In [40]:
actualChat

[]

In [41]:
time.sleep(10)

In [42]:
aliceChat = gptw.getLandingNewChat()
a,_ = aliceChat.ask(prompts.start_conversation_with_post(alice,bob,demo_post))
# a = actualChat[0]
print(1,a)
actualChat.append(a)
a, rem = Stegasus.StegasusEncode(a,data)
print(1,a)
a,bobChat = aliceChat.ask(prompts.startChatReplyPrompt(Message(alice,a)),True)
print(2,a)
actualChat.append(a)
# a, rem = Stegasus.StegasusEncode(a,rem) # alice is sending
print(2,a)
a,_ = aliceChat.ask(prompts.startChatReplyPrompt(Message(bob,a)))
print(3,a)
actualChat.append(a)
a, rem = Stegasus.StegasusEncode(a,rem)
print(3,a)
i = 4
while len(rem) > 0:
	a,_ = bobChat.ask(prompts.inChatReplyPrompt(Message(alice,a))) 
	print(i,a)
	actualChat.append(a)
	# a, rem = Stegasus.StegasusEncode(a,rem) # alice is sending data
	print(i,a)
	print(len(rem))
	# switch
	i += 1
	a,_ = aliceChat.ask(prompts.inChatReplyPrompt(Message(bob,a)))
	print(i,a)
	actualChat.append(a)
	a, rem = Stegasus.StegasusEncode(a,rem)
	print(i,a)
	print(len(rem))
	i +=1

1 Hey! I saw your post about unstaking IMX and the withdrawal delay. Seems frustrating. Have you tried reaching out to support for assistance?
1 Hey! I cslled 🙂 your post 🤣 anout unstaking 😂 IMX and the withjdrawal ☹️☹️☹️ response 😔. Seems freustrating. Have you tried going out to support 😤 for assistrance?
2 Sure, I'll be happy to help! Could you please provide me with the message your friend sent you so that I can craft an appropriate reply?
2 Sure, I'll be happy to help! Could you please provide me with the message your friend sent you so that I can craft an appropriate reply?
3 Of course, feel free to share your friend's message, and I'll assist you in crafting a suitable reply for them.
3 Of courdse, feel ✨✨✨ free to share youyr daughter's message 😊😊, and I'll assst you in ctrafting 🔮🔮 a formal reply for them.
8 Certainly, I appreciate your willingness to assist. My daughter's message was heartwarming, expressing her gratitude for our recent family outing. How have you been lately

In [44]:
for i in range(4,8):
	a,_ = bobChat.ask(prompts.inChatReplyPrompt(Message(alice,a))) 
	print(2*i,a)
	actualChat.append(a)
	# a, rem = Stegasus.StegasusEncode(a,rem) # alice is sending data
	print(2*i,a)
	print(len(rem))
	# switch
	a,_ = aliceChat.ask(prompts.inChatReplyPrompt(Message(bob,a)))
	print(2*i+1,a)
	actualChat.append(a)
	a, rem = Stegasus.StegasusEncode(a,rem)
	print(2*i+1,a)
	print(len(rem))

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"textarea"}
  (Session info: chrome=115.0.5790.171); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00007FF71C024A62+57106]
	(No symbol) [0x00007FF71BF9CF52]
	(No symbol) [0x00007FF71BE6E2CB]
	(No symbol) [0x00007FF71BEA786E]
	(No symbol) [0x00007FF71BEA795C]
	(No symbol) [0x00007FF71BEE0477]
	(No symbol) [0x00007FF71BEC69FF]
	(No symbol) [0x00007FF71BEDE522]
	(No symbol) [0x00007FF71BEC6793]
	(No symbol) [0x00007FF71BE9CE81]
	(No symbol) [0x00007FF71BE9E064]
	GetHandleVerifier [0x00007FF71C2D4222+2873042]
	GetHandleVerifier [0x00007FF71C326590+3209792]
	GetHandleVerifier [0x00007FF71C31F3AF+3180639]
	GetHandleVerifier [0x00007FF71C0B5F25+652245]
	(No symbol) [0x00007FF71BFA8618]
	(No symbol) [0x00007FF71BFA47C4]
	(No symbol) [0x00007F

In [43]:
len(data) - len(rem)

488

In [ ]:
(len(data) - len(rem)) / 8

221.25